In [7]:
import torch
import torch.nn as nn


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax
        return out


In [8]:
import nltk
import numpy as np
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    """
    sentence = ["hello, "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bag =   [0,     1,       0,   1,    0,      0,       0 ]
    """
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    
    bag = np.zeros(len(all_words), dtype = np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
import json
# from nltk_utils import tokenize, stem, bag_of_words
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# from model import NeuralNet

with open('intents.json', 'r') as f:
    intents = json.load(f)

# print(intents)

all_words = []
tags = []
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['?','!','.',',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags  = sorted(set(tags))

X_train  = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    
    label = tags.index(tag)
    y_train.append(label) # CrossEntropyLoss

X_train = np.array(X_train)
y_train = np.array(y_train)

class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train
        
    # dataset[idx]
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

# Hyper paramters
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001
num_epochs = 1000


dataset = ChatDataset()
train_loader = DataLoader(dataset = dataset, batch_size = batch_size,
                        shuffle =True, num_workers=2)

device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)

        # Forward
        outputs = model(words)
        loss = criterion(outputs, labels)

        # backward and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch +1) % 100 == 0:
        # f'epoch {epoch+1}/{num_epochs}, loss={loss.item():.4f}'
        print("epoch {}/{}, loss={:.4f}.".format(epoch+1,num_epochs,loss.item()))

# print(f'Final loss, loss={loss.item():.4f}')
print("Final Loss, loss{:.4f}".format(loss.item()))

epoch 100/1000, loss=1.0966.
epoch 200/1000, loss=0.1166.
epoch 300/1000, loss=0.0237.
epoch 400/1000, loss=0.0152.
epoch 500/1000, loss=0.0134.
epoch 600/1000, loss=0.0023.
epoch 700/1000, loss=0.0010.
epoch 800/1000, loss=0.0010.
epoch 900/1000, loss=0.0015.
epoch 1000/1000, loss=0.0013.
Final Loss, loss0.0013


In [10]:
data = {
    "model_state":model.state_dict(),
    "input_size":input_size,
    "output_size":output_size,
    "hiddent_size": hidden_size,
    "all_words":all_words,
    "tags": tags
}
FILE = "data.pth"
torch.save(data, FILE)

print("Traning complete. file saved to",FILE)

Traning complete. file saved to data.pth


In [11]:
import random
import json
import torch 

device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

with open('intents.json','r') as f:
  intents = json.load(f)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data['input_size']
hidden_size = data['hiddent_size']
output_size = data['output_size']
all_words = data['all_words']
tags = data['tags']
model_state = data['model_state']

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

cpu


NeuralNet(
  (l1): Linear(in_features=68, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=9, bias=True)
  (relu): ReLU()
)

In [ ]:
bot_name = "Taru"
print("Let's Chat! Type 'quit' to exit")

while True:
  sentence = input('You: ')
  if sentence =='quit':
    break
  
  sentence = tokenize(sentence)
  X = bag_of_words(sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X)

  output = model(X)
  _, predicted = torch.max(output, dim=1)
  tag = tags[predicted.item()]

  probs = torch.softmax(output, dim=1)
  prob = probs[0][predicted.item()]


  if prob.item() >0.75:
    for intent in intents["intents"]:
      if tag == intent["tag"]:
        print(bot_name, ":",random.choice(intent['responses']))
        # print("{}:{}".format(bot_name,random.choice(intent['responses'])))
      
  else:
    print("{} I do not understand...".format(bot_name))

Let's Chat! Type 'quit' to exit
You: Hi
Taru : Hi there, what can I do for you?
You: thanks
Taru : My Pleasure
You: time
Taru : Bye! Come back again soon.
You: delivery time
TaruI do not understand...
You: are you accept cash
Taru : We accept most major Credit cards, Master Card, Paypal, American Express and Discover.
You: who are you
Taru : Hi there, how can I help?
You: name
TaruI do not understand...
You: what is your name
Taru : You can call me Tarun.
You: thanks 
Taru : Happy to help!
You: what you sell
TaruI do not understand...
You: what do you sell
Taru : We sell Cake, Flowers, Gifts and delivery across 2500+ cities in India!
You: i want to buy cake
Taru : We sell Cake, Flowers, Gifts and delivery across 2500+ cities in India!
